In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime
from sklearn.linear_model import LogisticRegression
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import cross_validate
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, roc_auc_score, log_loss
from sklearn.compose import ColumnTransformer

In [3]:
data = pd.read_csv('data.csv')
data.head()

,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,impressions,clicks
0,2021-09-27 00:01:30.000000,0,0,5664530014561852622,0,0,0,1240,0.067,0.035016,-7.268846,0,0.010,0.049516,-5.369901,1,1
1,2021-09-26 22:54:49.000000,1,1,5186611064559013950,0,0,1,1,0.002,0.054298,-2.657477,269,0.004,0.031942,-4.449220,1,1
2,2021-09-26 23:57:20.000000,2,2,2215519569292448030,3,0,0,2,0.014,0.014096,-3.824875,21,0.014,0.014906,-3.939309,1,1
3,2021-09-27 00:04:30.000000,3,3,6262169206735077204,0,1,1,3,0.012,0.015232,-3.461357,99,0.006,0.050671,-3.418403,1,1
4,2021-09-27 00:06:21.000000,4,4,4778985830203613115,0,1,0,4,0.019,0.051265,-4.009026,11464230,6.790,0.032005,-2.828797,1,1


Препроцессинг из первого дз + прислушился к комментариям и добавил несколько интеракций.

In [4]:
def data_preprocess(data):
    data['date_time'] = pd.to_datetime(data['date_time'], errors='coerce')
    data['date'] = data['date_time'].dt.date
    data['day_of_the_week'] = data['date_time'].dt.dayofweek
    data['day_period'] = (data['date_time'].dt.hour % 24 + 4) // 4
    data = data[(data['g0'] >= 0) & (data['g1'] >= 0)]

    mask_first = (data['date_time'].dt.date != datetime.date(2021, 10, 2)) & (data['banner_id'] == data['banner_id0'])
    mask_last = (data['date_time'].dt.date == datetime.date(2021, 10, 2)) & (data['banner_id'] == data['banner_id0'])
    
    first = data[mask_first].copy()
    last = data[mask_last].dropna(subset=['coeff_sum0', 'coeff_sum1']).copy()

    data.drop(['date_time'], axis=1, inplace=True)
    
    for i in ['os_id', 'country_id']:
        for j in ['banner_id']:
            inter_feature = f"{i}_{j}"
            first[inter_feature] = first[i].astype(str) + '_' + first[j].astype(str)
            last[inter_feature] = last[i].astype(str) + '_' + last[j].astype(str)
    
    return data, first, last

In [5]:
data,first,last = data_preprocess(data)

/var/folders/nh/5_1n3ycs0f3g8ltxlc_pcr6m0000gn/T/ipykernel_19025/2016565790.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(['date_time'], axis=1, inplace=True)


One-hot encoding

In [6]:
def transform_data(first, non_banner_features):
    transforms = ColumnTransformer([
        ('banner_one_hot', OneHotEncoder(sparse=True, handle_unknown='ignore'), [0]),
        ('others_one_hot', OneHotEncoder(sparse=True, handle_unknown='ignore'), list(range(1, len(non_banner_features) + 1))),
    ])
    
    selected_columns = first[['banner_id'] + non_banner_features]
    transforms.fit(selected_columns.to_numpy())
    return transforms


In [7]:
features = ['day_of_the_week', 'day_period','zone_id', 'os_id', 'country_id' ,'country_id_banner_id','os_id_banner_id']
transforms = transform_data(first, features)


Деление датасета

In [8]:
X_train = transforms.transform(first[['banner_id'] + features].to_numpy())
X_test0 = transforms.transform(last[['banner_id0'] + features].to_numpy())
X_test1 = transforms.transform(last[['banner_id1'] + features].to_numpy())

лучшее значение C=1.0 из ДЗ 1

In [9]:
model = LogisticRegression(solver='liblinear', C=1.0, random_state=3141,verbose=1)
model.fit(X_train, first['clicks'])

[LibLinear]iter  1 act 6.076e+06 pre 5.405e+06 delta 1.608e+00 f 8.347e+06 |g| 7.677e+06 CG   1
iter  2 act 7.364e+05 pre 6.102e+05 delta 1.608e+00 f 2.271e+06 |g| 1.726e+06 CG   1
iter  3 act 1.837e+05 pre 1.519e+05 delta 1.608e+00 f 1.534e+06 |g| 5.154e+05 CG   2
cg reaches trust region boundary
iter  4 act 5.808e+04 pre 5.179e+04 delta 1.998e+00 f 1.351e+06 |g| 1.477e+05 CG   3
cg reaches trust region boundary
iter  5 act 2.352e+04 pre 2.226e+04 delta 2.433e+00 f 1.293e+06 |g| 5.143e+04 CG   3
cg reaches trust region boundary
iter  6 act 1.420e+04 pre 1.348e+04 delta 2.680e+00 f 1.269e+06 |g| 2.660e+04 CG   5
cg reaches trust region boundary
iter  7 act 1.005e+04 pre 9.691e+03 delta 2.939e+00 f 1.255e+06 |g| 2.064e+04 CG   6
cg reaches trust region boundary
iter  8 act 6.097e+03 pre 6.362e+03 delta 2.939e+00 f 1.245e+06 |g| 1.487e+04 CG   8
cg reaches trust region boundary
iter  9 act 4.673e+03 pre 4.599e+03 delta 3.013e+00 f 1.239e+06 |g| 1.157e+04 CG   9
cg reaches trust region bo

LogisticRegression(random_state=3141, solver='liblinear', verbose=1)

In [12]:
p_0 = model.predict_proba(X_test0)[:, 1]
p_1 = model.predict_proba(X_test1)[:, 1]

In [13]:
print(f'log_loss: {log_loss(last["clicks"], p_0)}')
print(f'auc: {roc_auc_score(last["clicks"], p_0)}')

log_loss: 0.13450044509454878
auc: 0.7903352370721316
log_loss: 0.13568115622718738
auc: 0.7826085328668043


Оценка вероятности

$X\sim N(\mu_1,\sigma_1^2)\\$
$Y\sim N(\mu_2,\sigma_1^2)\\$
$D=X-Y\\$
$D\sim N(\mu_1-\mu_2,\sigma_1^2+\sigma_2^2)\\$
$F$ фун-я распределения

In [116]:
from scipy.stats import norm
from scipy.special import logit


import numpy as np
from scipy.stats import norm

def calculate(last, p_0, p_1, LAMBDA=10):
    pi_0 = norm.cdf(
        (last['coeff_sum0'] - last['coeff_sum1']) / 
        np.sqrt(last['g0'] ** 2 + last['g1'] ** 2))

    pi_1 = norm.cdf(
        (logit(p_0) - logit(p_1)) / 
        np.sqrt(last['g0'] ** 2 + last['g1'] ** 2))

    mask = np.isfinite(pi_0) * np.isfinite(pi_1) * (abs(pi_0) > np.finfo(np.float64).tiny)
    
    filtered_pi_0 = pi_0[mask]
    filtered_pi_1 = pi_1[mask]
    
    V_cips = np.mean(last['clicks'][mask] * np.minimum(filtered_pi_1 / filtered_pi_0, LAMBDA))
    print(f'V_cips = {V_cips:.3f} при lambda = {LAMBDA}')
    
    return filtered_pi_0, filtered_pi_1, V_cips


In [117]:
filtered_pi_0, filtered_pi_1, V_cips = calculate(last,p_0,p_1)

V_cips = 0.069 при lambda = 10
